TỪ TIẾNG VIỆT SANG TIẾNG ANH

## Giai đoạn 1: Chuẩn bị dữ liệu

1. Thu thập dữ liệu


In [17]:
pip install pyvi # Thư viện dùng riêng tách từ tiếng Việt

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from pyvi import ViTokenizer  # Thư viện chuyên dùng để tách từ tiếng Việt
import os

In [19]:
# Cấu hình thiết bị & Hyperparameters
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang chạy trên: {DEVICE}")

# Đổi đường dẫn này đúng với nơi bạn lưu file trên máy
DATA_DIR = r"D:\Period5\NLP\BTL\Data" 
TRAIN_SRC = os.path.join(DATA_DIR, "train.vi.txt")
TRAIN_TRG = os.path.join(DATA_DIR, "train.en.txt")

# Thông số huấn luyện
BATCH_SIZE = 128        # Số câu xử lý cùng lúc
MAX_LEN = 128          # Độ dài câu tối đa
FREQ_THRESHOLD = 2     # Bỏ qua các từ xuất hiện dưới 2 lần

Đang chạy trên: cpu


2. Tiền xử lý, Xây dựng từ điển

Thách thức:
Tiếng Anh dùng khoảng trắng để tách từ (*student*), nhưng tiếng Việt có nhiều từ ghép (*sinh viên*, *đất nước*). Nếu chỉ tách bằng khoảng trắng, từ **"sinh viên"** sẽ bị tách thành **"sinh"** (birth) và **"viên"** (pill/member), làm sai lệch nghĩa.

**Giải pháp:**  
Bắt buộc dùng **tokenizer chuyên dụng** như **PyVi** (dùng thuật toán n-grams) hoặc **VnCoreNLP**.

**Ví dụ:**  
`"Học máy rất thú vị"` → `["Học_máy", "rất", "thú_vị"]`


In [20]:
# --- SỬA LẠI CELL 2: GIỚI HẠN TỪ VỰNG (QUAN TRỌNG) ---

class Vocabulary:
    def __init__(self):
        self.itos = {0: "<pad>", 1: "<sos>", 2: "<eos>", 3: "<unk>"}
        self.stoi = {"<pad>": 0, "<sos>": 1, "<eos>": 2, "<unk>": 3}

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_vi(text):
        return ViTokenizer.tokenize(text).split()

    @staticmethod
    def tokenizer_en(text):
        return text.lower().replace(".", "").replace(",", "").replace("?", "").split()

    def build_vocabulary(self, sentence_list, lang="vi", max_size=15000): # <--- THÊM max_size
        frequencies = Counter()
        tokenizer = self.tokenizer_vi if lang == "vi" else self.tokenizer_en

        print(f"⏳ Đang đếm tần suất từ cho {lang}...")
        for sentence in sentence_list:
            frequencies.update(tokenizer(sentence))
        
        # Chỉ lấy top từ phổ biến nhất (trừ đi 4 token đặc biệt)
        target_vocab_size = max_size - 4
        common_words = frequencies.most_common(target_vocab_size)
        
        idx = 4
        for word, freq in common_words:
            self.stoi[word] = idx
            self.itos[idx] = word
            idx += 1
            
    def numericalize(self, text, lang="vi"):
        tokenizer = self.tokenizer_vi if lang == "vi" else self.tokenizer_en
        tokenized_text = tokenizer(text)
        return [self.stoi.get(token, self.stoi["<unk>"]) for token in tokenized_text]

print("⏳ Đang đọc dữ liệu và xây từ điển...")
try:
    with open(TRAIN_SRC, "r", encoding="utf-8", errors="ignore") as f: src_txt = f.readlines()
    with open(TRAIN_TRG, "r", encoding="utf-8", errors="ignore") as f: trg_txt = f.readlines()

    # GIỚI HẠN SỐ LƯỢNG TỪ (15000 là con số an toàn cho máy cá nhân)
    src_vocab = Vocabulary()
    src_vocab.build_vocabulary(src_txt, "vi", max_size=15000)
    
    trg_vocab = Vocabulary()
    trg_vocab.build_vocabulary(trg_txt, "en", max_size=15000)

    print(f"✅ Từ vựng Việt (Đã cắt giảm): {len(src_vocab)}")
    print(f"✅ Từ vựng Anh (Đã cắt giảm): {len(trg_vocab)}")
    
    torch.save(src_vocab, "checkpoints/src_vocab.pth")
    torch.save(trg_vocab, "checkpoints/trg_vocab.pth")

except FileNotFoundError:
    print("❌ LỖI: Không tìm thấy file dữ liệu!")

⏳ Đang đọc dữ liệu và xây từ điển...
⏳ Đang đếm tần suất từ cho vi...
⏳ Đang đếm tần suất từ cho en...
✅ Từ vựng Việt (Đã cắt giảm): 15000
✅ Từ vựng Anh (Đã cắt giảm): 15000


In [21]:
# #Test vocalabulary với dữ liệu giả


# # Tạo dữ liệu giả (Giả lập file text)
# # Lưu ý: "sinh viên" xuất hiện 2 lần -> Sẽ được đưa vào từ điển
# # "học máy" xuất hiện 1 lần -> Sẽ bị loại bỏ (thành <unk>)
# sentences = [
#     "tôi là sinh viên đại học",
#     "sinh viên cần học chăm chỉ",
#     "học máy rất thú vị"
# ]

# # 3. Khởi tạo và xây từ điển
# print("--- ĐANG TEST VỚI DỮ LIỆU GIẢ ---")
# vocab_test = Vocabulary()
# vocab_test.build_vocabulary(sentences, lang="vi")

# # 4. Kiểm tra kết quả
# print(f"Tổng số từ trong từ điển: {len(vocab_test)}")
# print("Danh sách từ và ID (stoi):", vocab_test.stoi)

# # 5. Test thử biến đổi câu sang số (Numericalize)
# cau_test = "sinh viên học máy"
# input_ids = vocab_test.numericalize(cau_test, lang="vi")

# print(f"\nCâu gốc: '{cau_test}'")
# print(f"Tokenized: {Vocabulary.tokenizer_vi(cau_test)}")
# print(f"Chuyển sang số: {input_ids}")

# # Giải thích kết quả mong đợi:
# # - "sinh_viên" (có trong stoi) -> Ra số ID cụ thể (ví dụ 4)
# # - "học_máy" (tần suất < 2, không có trong stoi) -> Ra số 3 (<unk>)

In [22]:
# --- CELL 3: DATASET & DATALOADER ---

class TranslationDataset(Dataset):
    def __init__(self, src_lines, trg_lines, src_vocab, trg_vocab):
        self.src_lines = src_lines
        self.trg_lines = trg_lines
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.src_lines)

    def __getitem__(self, index):
        src_text = self.src_lines[index].strip()
        trg_text = self.trg_lines[index].strip()

        # 1. Chuyển chữ sang số
        src_token_ids = self.src_vocab.numericalize(src_text, "vi")
        trg_token_ids = self.trg_vocab.numericalize(trg_text, "en")

        # 2. QUAN TRỌNG: Cắt bớt nếu câu quá dài (Tránh lỗi IndexError)
        # MAX_LEN - 2 vì phải chừa chỗ cho <sos> và <eos>
        if len(src_token_ids) > MAX_LEN - 2:
            src_token_ids = src_token_ids[:MAX_LEN - 2]
        
        if len(trg_token_ids) > MAX_LEN - 2:
            trg_token_ids = trg_token_ids[:MAX_LEN - 2]

        # 3. Thêm <sos> và <eos>
        src_encoded = [1] + src_token_ids + [2]
        trg_encoded = [1] + trg_token_ids + [2]

        return torch.tensor(src_encoded), torch.tensor(trg_encoded)

class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        src = [item[0] for item in batch]
        trg = [item[1] for item in batch]
        src = pad_sequence(src, batch_first=True, padding_value=self.pad_idx)
        trg = pad_sequence(trg, batch_first=True, padding_value=self.pad_idx)
        return src, trg

print("⏳ Đang tạo DataLoader...")
train_dataset = TranslationDataset(src_txt, trg_txt, src_vocab, trg_vocab)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=MyCollate(pad_idx=0)
)
print(f"✅ Dữ liệu sẵn sàng! Tổng {len(train_loader)} batches.")

⏳ Đang tạo DataLoader...
✅ Dữ liệu sẵn sàng! Tổng 3907 batches.


In [23]:
#Lưu bộ từ điển đã tạo ra thành file .pth để dùng lại sau này

import torch
import os

# Tạo thư mục checkpoints nếu chưa có để lưu cho gọn
if not os.path.exists("checkpoints"):
    os.makedirs("checkpoints")

print("Đang lưu bộ từ điển...")

# 1. Lưu từ điển Tiếng Việt (Source)
torch.save(src_vocab, "checkpoints/src_vocab.pth")

# 2. Lưu từ điển Tiếng Anh (Target)
torch.save(trg_vocab, "checkpoints/trg_vocab.pth")

print("-> Đã lưu thành công 2 file 'src_vocab.pth' và 'trg_vocab.pth' vào thư mục checkpoints!")

Đang lưu bộ từ điển...
-> Đã lưu thành công 2 file 'src_vocab.pth' và 'trg_vocab.pth' vào thư mục checkpoints!


In [24]:
# Load lại
src_vocab = torch.load("checkpoints/src_vocab.pth")
trg_vocab = torch.load("checkpoints/trg_vocab.pth")

print("Đã load lại từ điển thành công!")
print(f"Việt: {len(src_vocab)} từ")
print(f"Anh: {len(trg_vocab)} từ")

Đã load lại từ điển thành công!
Việt: 15000 từ
Anh: 15000 từ


## Giai đoạn 2: Xây dựng transformer

1. Xây dựng Cơ chế Self-Attention

$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$


In [25]:
# --- CELL 2: SELF-ATTENTION ---
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads # Ví dụ: 256 / 8 = 32

        assert (self.head_dim * heads == embed_size), "Embed size phải chia hết cho heads"

        # 3 ma trận trọng số để tạo ra Query, Key, Value
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0] # Batch size
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # 1. Chia embedding thành các đầu (heads) nhỏ
        # Shape: (N, seq_len, heads, head_dim)
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        # 2. Chiếu qua Linear
        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # 3. Tính điểm tương đồng (Dot Product) giữa Query và Key
        # "nqhd,nkhd->nhqk": Nhân ma trận theo chiều head_dim
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        # 4. Masking (Che đi các từ không được nhìn thấy)
        if mask is not None:
            # Gán giá trị cực nhỏ (-1e20) vào vị trí padding hoặc tương lai
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # 5. Softmax: Biến điểm số thành xác suất (0-1)
        # Chia căn bậc 2 (embed_size ** 0.5) để ổn định số học
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        # 6. Nhân xác suất với Value để lấy thông tin
        out = torch.einsum("nhqk,nvhd->nqhd", [attention, values])
        
        # 7. Nối lại thành vector ban đầu
        out = out.reshape(N, query_len, self.heads * self.head_dim)
        out = self.fc_out(out)
        
        return out

In [26]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        # Attention -> Add & Norm -> FeedForward -> Add & Norm
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

2. Encoder

In [27]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        # Tạo vector vị trí [0, 1, 2...]
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)
        return out

3. Decoder

In [28]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        # 1. Masked Attention (Tự nhìn bản thân, nhưng che tương lai)
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        
        # 2. Cross Attention (Nhìn sang Encoder để lấy thông tin nguồn)
        out = self.transformer_block(value, key, query, src_mask)
        return out

class Decoder(nn.Module):
    def __init__(self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size, heads, forward_expansion, dropout, device) for _ in range(num_layers)]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out

4. Lắp ráp thành transformer toàn diện


In [29]:
# --- CELL 4: FULL TRANSFORMER ---

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=256, num_layers=3, forward_expansion=4, heads=8, dropout=0.1, device="cpu", max_length=128):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length)
        self.decoder = Decoder(trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length)
        
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        # Mask padding: [Batch, 1, 1, Seq_Len]
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        # Mask tam giác (Look-ahead): [Batch, 1, Seq_Len, Seq_Len]
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # Chạy Encoder
        enc_src = self.encoder(src, src_mask)
        
        # Chạy Decoder
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

## Giai đoạn 3: Huấn luyện

In [ ]:
model = Transformer(
    len(src_vocab), len(trg_vocab),
    src_vocab.stoi["<pad>"], trg_vocab.stoi["<pad>"],
    device=DEVICE
).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=3e-4)
# NOTE: ignore padding của target (trg), không phải src
criterion = nn.CrossEntropyLoss(ignore_index=trg_vocab.stoi["<pad>"])

NUM_EPOCHS = 10
CLIP = 1.0  # gradient clipping
print(f"Bắt đầu huấn luyện {NUM_EPOCHS} epochs...")

best_loss = float('inf')
loss_history = []

try:
    for epoch in range(NUM_EPOCHS):
        model.train()            # rất quan trọng: bật chế độ train()
        epoch_loss = 0.0
    
        for batch_idx, (src, trg) in enumerate(train_loader):
            # chuyển device
            src = src.to(DEVICE)
            trg = trg.to(DEVICE)

            # forward: đưa input cho decoder là trg[:, :-1] (bỏ token cuối EOS)
            output = model(src, trg[:, :-1])

            # output: (batch, trg_len-1, vocab_size) -> reshape để tính loss
            output = output.reshape(-1, output.shape[2])           # (batch*(trg_len-1), vocab_size)
            target = trg[:, 1:].reshape(-1)                        # (batch*(trg_len-1),)

            optimizer.zero_grad()
            loss = criterion(output, target)
            loss.backward()

            # optional: gradient clipping để ổn định
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

            optimizer.step()

            epoch_loss += loss.item()

            # optional: in progress từng vài batch để debug
            if (batch_idx + 1) % 100 == 0:
                print(f" Epoch {epoch+1} | Batch {batch_idx+1}/{len(train_loader)} | batch-loss: {loss.item():.4f}")

        # tính loss trung bình epoch
        avg_loss = epoch_loss / len(train_loader)
        loss_history.append(avg_loss)

        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Loss: {avg_loss:.4f}")

        # lưu model nếu tốt hơn
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), "checkpoints/best_transformer.pth")
            print(f"| -> Đã lưu Kỷ Lục Mới! (Saved Best Model)")
        else:
            print("|")

except Exception as e:
    # nếu có lỗi runtime, in traceback đầy đủ để bạn biết nguyên nhân
    print("Đã xảy ra lỗi trong quá trình huấn luyện:")
    traceback.print_exc()

Bắt đầu huấn luyện 10 epochs...


## Giai đoạn 4: Suy luận

## Giai đoạn 5: Đánh giá